# Masked key byte extractor

### Append a masked key byte column to the ASCAD file.

In [1]:
import h5py
from src.data.loader.ascad import ASCADData, ASCAD_DATA, ASCAD_DATA_VAR, ASCADDataType
from tqdm import tqdm
import numpy as np

ascad = ASCADData.random_key()
masked_byte_ix = 2
full_key_ix = 1

pm = ascad["Profiling_traces"]["metadata"]
am = ascad["Attack_traces"]["metadata"]

pml, aml = np.zeros(len(pm), dtype=np.uint8), np.zeros(len(am), dtype=np.uint8)

for ix in tqdm(range(len(pm))):
    pml[ix] = pm[ix][full_key_ix][masked_byte_ix]

for ix in tqdm(range(len(am))):
    aml[ix] = am[ix][full_key_ix][masked_byte_ix]

ascad.close()

100%|██████████| 100000/100000 [00:12<00:00, 8049.14it/s]


In [2]:
pml

array([173,  17, 199, ..., 205, 146, 189], dtype=uint8)

In [3]:
aml

array([34, 34, 34, ..., 34, 34, 34], dtype=uint8)

In [4]:
wrt = h5py.File(f"{ASCAD_DATA}{ASCAD_DATA_VAR}/{ASCADDataType.default}.h5", 'r+')

for grp_name, grp_l in [("Profiling_traces", pml), ("Attack_traces", aml)]:
    grp = wrt[grp_name]

    if "labels_mask" in grp:
        del grp["labels_mask"]
    grp["labels_mask"] = grp_l

wrt.close()

chk = h5py.File(f"{ASCAD_DATA}{ASCAD_DATA_VAR}/{ASCADDataType.default}.h5", 'r')
print(list(chk["Profiling_traces"]))
chk.close()

['labels', 'labels_mask', 'metadata', 'traces']


In [5]:
ascad = ASCADData.random_key()
ascad["Profiling_traces"]["labels_mask"][0]

173